In [1]:
import mediapipe as mp
import cv2 as cv
import numpy as np
import time
import os

mpPose = mp.solutions.pose
Pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

video_path = os.path.join(os.curdir,'video','data')
videos = os.listdir(video_path)

In [2]:
from tensorflow import keras
model = keras.models.load_model('models/resistance_band_detector2.h5')

In [3]:
def predict(frame,landmarks):
    #foot
#     xs = landmarks[26].x,landmarks[25].x,landmarks[30].x,landmarks[29].x
#     ys = landmarks[26].y,landmarks[25].y,landmarks[30].y,landmarks[29].y
    #hip
    xs = landmarks[24].x,landmarks[23].x,landmarks[26].x,landmarks[25].x
    ys = landmarks[24].y,landmarks[23].y,landmarks[26].y,landmarks[25].y
    xmin,xmax = min(xs),max(xs)
    ymin,ymax = min(ys),max(ys)
    height,width = frame.shape[:2]
    ymin,ymax = int(ymin*height),int(ymax*height)
    xmin,xmax = int(xmin*width),int(xmax*width)
    input_x = frame[ymin:ymax,xmin:xmax,:]
    input_x = cv.resize(input_x,(80,70))
    input_x = input_x/255
    pred = model.predict(input_x[np.newaxis,...])
    pred = np.argmax(pred[0])
    if pred == 0:
        label = 'no band'
        frame = cv.rectangle(frame,(xmin,ymin),(xmax,ymax),(0,0,205),2)
        cv.putText(frame,label,(xmax,ymax+5),cv.FONT_HERSHEY_COMPLEX,.6,(0,0,205),1)
    elif pred == 1:
        label = 'yes band'
        frame = cv.rectangle(frame,(xmin,ymin),(xmax,ymax),(0,205,0),2)
        cv.putText(frame,label,(xmax,ymax+5),cv.FONT_HERSHEY_COMPLEX,.6,(0,205,0),1)
    return frame

In [6]:
vi_path = os.path.join(os.curdir,'video','data')
vi_path = 'C:/Users/atp17/Desktop/work/video'
vids = os.listdir(vi_path)
print(vids)
for video in vids[1:]:
    file = os.path.join(vi_path,video)
    cam = cv.VideoCapture(file)
    ptime = 0
    while True:
        status,frame = cam.read()
        if status:
            frame_rgb = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
            processed = Pose.process(frame_rgb)
            if processed.pose_landmarks:
                frame = predict(frame,processed.pose_landmarks.landmark)
            ctime = time.time()
            fps = int(1/(ctime-ptime))
            cv.putText(frame,'frame: '+str(fps),(10,30),cv.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
            ptime = ctime
            cv.imshow('video',frame)
            if cv.waitKey(1)&0xff==ord('q'):
                cam.release()
                cv.destroyAllWindows()
                break
        else:
            cam.release()
            cv.destroyAllWindows()
            break

['VID-20220415-WA0002.mp4', 'VID-20220415-WA0003.mp4', 'VID-20220415-WA0004.mp4', 'VID-20220415-WA0005.mp4', 'VID-20220415-WA0006.mp4', 'VID-20220415-WA0007.mp4', 'VID-20220415-WA0008.mp4', 'VID-20220415-WA0009.mp4', 'VID-20220415-WA0010.mp4', 'VID-20220415-WA0011.mp4', 'VID-20220415-WA0012.mp4', 'VID-20220415-WA0013.mp4', 'VID-20220415-WA0014.mp4', 'VID-20220415-WA0015.mp4']
1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


In [20]:
cam = cv.VideoCapture(0)
ptime = 0
while True:
    status,frame = cam.read()
    if status:
        frame_rgb = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
        processed = Pose.process(frame_rgb)
        if processed.pose_landmarks:
            frame = predict(frame,processed.pose_landmarks.landmark)
        ctime = time.time()
        fps = int(1/(ctime-ptime))
        cv.putText(frame,'frame: '+str(fps),(10,30),cv.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
        ptime = ctime
        cv.imshow('video',frame)
        if cv.waitKey(1)&0xff==ord('q'):
            cam.release()
            cv.destroyAllWindows()
            break
    else:
        cam.release()
        cv.destroyAllWindows()
        break

1/1 [==============================] - 0s 40ms/step


error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [21]:
cv.destroyAllWindows()

### saving video

In [18]:
vi_path = os.path.join(os.curdir,'video','data')
vids = os.listdir(vi_path)
for i,video in enumerate(vids):
    file = os.path.join(vi_path,video)
    cam = cv.VideoCapture(file)
    
    frame_width = int(cam.get(3))
    frame_height = int(cam.get(4))
    size = (frame_width, frame_height)
    result = cv.VideoWriter(str(i)+'.mp4', 
                         cv.VideoWriter_fourcc(*'MP4V'),
                         10, size)
    
    ptime = 0
    while True:
        status,frame = cam.read()
        if status:
            frame_rgb = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
            processed = Pose.process(frame_rgb)
            if processed.pose_landmarks:
                frame = predict(frame,processed.pose_landmarks.landmark)
            ctime = time.time()
            fps = int(1/(ctime-ptime))
            cv.putText(frame,'fps: '+str(fps),(10,30),cv.FONT_HERSHEY_COMPLEX,1,(0,255,255),2)
            ptime = ctime
            result.write(frame)
            cv.imshow('video',frame)
            if cv.waitKey(1)&0xff==ord('q'):
                result.release()
                cam.release()
                cv.destroyAllWindows()
                break
        else:
            result.release()
            cam.release()
            cv.destroyAllWindows()
            break

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 22ms/step
